<a href="https://colab.research.google.com/github/jonhyuk0922/HackerTon/blob/master/%5BDACON%5D%EA%B8%B0%ED%9B%84%EA%B8%B0%EC%88%A0%EB%B6%84%EB%A5%98_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. 데이터 및 라이브러리 불러오기**

참고 코드: 텐서플로2와 머신러닝으로 시작하는 자연어처리(위키북스)

https://github.com/NLP-kr/tensorflow-ml-nlp-tf2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 450kB 39.3MB/s 
     |████████████████████████████████| 92kB 13.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
!pip install transformers
from transformers import TFBertModel
from transformers import BertModel
BertModel.from_pretrained # good to go

     |████████████████████████████████| 2.5MB 9.1MB/s 
     |████████████████████████████████| 901kB 55.0MB/s 
     |████████████████████████████████| 3.3MB 59.9MB/s 


<bound method PreTrainedModel.from_pretrained of <class 'transformers.models.bert.modeling_bert.BertModel'>>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import json
import os
import tqdm

from konlpy.tag import Okt

import sklearn
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import log_loss, accuracy_score,f1_score
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
import zipfile

local_zip = '/content/drive/MyDrive/HackerTon/데이콘_기후기술분류/open.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('./')
zip_ref.close()

In [ ]:
train=pd.read_csv('/content/open/train.csv')
test=pd.read_csv('/content/open/test.csv')
sample_submission=pd.read_csv('/content/open/sample_submission.csv')

In [ ]:
print(f'train.shape:{train.shape}')
print(f'test.shape:{test.shape}')
print(f'train label 개수: {train.label.nunique()}')

train.shape:(174304, 13)
test.shape:(43576, 12)
train label 개수: 46


# **2. 데이터 전처리**

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="0" #환경변수 설정

In [ ]:
train.tail()

,index,제출년도,사업명,사업_부처명,계속과제여부,내역사업명,과제명,요약문_연구목표,요약문_연구내용,요약문_기대효과,요약문_한글키워드,요약문_영문키워드,label
174299,174299,2016,개인연구지원,미래창조과학부,신규,중견연구,혈관내막 증식을 치료하기 위한 표적분자 발굴,"증식성 혈관질환과 염증반응과의 관련성이 높으나, 혈관평활근세포의 탈분화를 유도하는 ...",● 1차년: 혈관내막 증식과 관련된 표적분자 발굴 및 염증세포의 활성과 분화에 대한...,심혈관 질환 발생의 새로운 기전 규명뿐만 아니라 제어할 수 있는 새로운 방법을 탐...,"혈관내막 증식,혈관 개조,혈관평활근세포,염증세포,혈관염증,혈관평활근세포 탈분화,표적분자","Intimahyperplasia,Vascularremodeling,Vasculars...",0
174300,174300,2016,개인연구지원,미래창조과학부,신규,중견연구,삼킴곤란 개선을 위한 물성조절 식품시스템에서의 유동층과립 복합소재의 유변물성 변화 ...,삼킴곤란을 겪고 있는 고령자의 안전한 식품섭취는 삼킴에 적합한 유변물성을 가진 물성...,본 연구에서는 여러 가지 유변물성 측정기술을 이용하여 고령자의 삼킴곤란 개선을 위해...,○ 그 동안 연구되지 않은 다양한 물성조절 검복합 소재의 과립제형 및 유변물성 연구...,"삼킴곤란,유변물성,유동층과립,물리적 특성,검복합물","Swallowingdifficulty,Rheologicalproperty,Fludi...",0
174301,174301,2018,이공학학술연구기반구축(R&D),교육부,계속,이공분야학문후속세대양성_대통령POST_DOC펠로우쉽,식물 유전체 정보 고도화 및 핵심 유전자 군 진화연구,"지난 10년여 동안 100여종에 대한 식물 유전체 정보가 완성되어 공개되었지만, 애...",○ 공개된 식물 유전체 정보 및 대상 유전자 군 선발\n\n - 공개된 식물 유...,- 본 과제에서 개발한 annotation tool은 유전체 고도화 부분에서 가장 ...,"유전자 정보 고도화,대상 유전자 군 재분석,비교 유전체 진화 분석,공개 식물 유전체...","Gene model improvement,Target gene family re-a...",0
174302,174302,2019,수요자맞춤형육종자원대량신속발굴기술개발(R&D),농촌진흥청,신규,유용형질 대량 특성평가를 통한 육종모본 발굴,"콩 유전자원 내건성, 내습성 대량평가 체계구축 및 자원선발",1) 내건 및 내습성 유전자원 수집\n\n2) 내건성 자원평가 체계구축\n\n3) ...,[1차 년도(2019)] \n\n1) 내건 및 내습성 유전자원 수집\n\n○ 기존 ...,○ 내건성 및 내습성 특성평가 체계구축 및 우수자원 발굴로 내재해성 콩 품종육성 자...,"콩, 가공적성, 병 저항성, 내재해성, 콩모자이크바이러스","soybean, food application, resistance, biotic-...",23
174303,174303,2019,이공학학술연구기반구축(R&D),교육부,신규,이공분야학문후속세대양성_박사후국내연수,광학적 위상 제어 기법을 적용한 디지털 홀로그래피 현미경 시스템 개발,본 연구의 목적은 이터븀이 도핑된 특수 광섬유를 이용하여 광학적인 방법으로 위상(P...,◦ 1차년도 연구개발 내용\n : 광섬유 비선형 굴절률 변화 효과를 이용한 광학...,본 연구를 통해 확보하고자 하는 안정적으로 위상 특성 제어를 통해 DHM 시스템을 ...,"광섬유,위상장금,위상이동,간섭계,디지털홀로그래피,비축,복원,이터듐","fiber,phase locking,phase shift,interferometer...",0


In [ ]:
train.columns

Index(['과제명', '요약문_연구목표', 'label'], dtype='object')

In [ ]:
#이번 베이스라인에서는 과제명 뿐만 아니라 요약문_연구내용도 모델에 학습시켜보겠습니다.
train=train[['과제명', '요약문_연구목표','요약문_한글키워드','label']]
test=test[['과제명', '요약문_연구목표','요약문_한글키워드']]
train['요약문_연구목표'].fillna('NAN', inplace=True)
test['요약문_연구목표'].fillna('NAN', inplace=True)
train['요약문_한글키워드'].fillna('NAN', inplace=True)
test['요약문_한글키워드'].fillna('NAN', inplace=True)

In [ ]:
train['data']=train['과제명']+train['요약문_연구목표']+train['요약문_한글키워드']
test['data']=test['과제명']+test['요약문_연구목표']+test['요약문_한글키워드']

In [ ]:
print(train.shape)
print(test.shape)

(174304, 5)
(43576, 4)


In [ ]:
train.info()
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174304 entries, 0 to 174303
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   과제명        174304 non-null  object
 1   요약문_연구목표   174304 non-null  object
 2   요약문_한글키워드  174304 non-null  object
 3   label      174304 non-null  int64 
 4   data       174304 non-null  object
dtypes: int64(1), object(4)
memory usage: 6.6+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43576 entries, 0 to 43575
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   과제명        43576 non-null  object
 1   요약문_연구목표   43576 non-null  object
 2   요약문_한글키워드  43576 non-null  object
 3   data       43576 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [ ]:
type(train['label'][3])

numpy.int64

In [ ]:
train.head(2)

,과제명,요약문_연구목표,요약문_한글키워드,label,data
0,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,○ 새로운 해충분류군의 동정기술 개발 및 유입확산 추적,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체",24,유전정보를 활용한 새로운 해충 분류군 동정기술 개발○ 새로운 해충분류군의 동정기술 ...
1,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...,최종목표: TRAIL 감수성 표적 유전자를 발굴하고 내성제어 기전을 연구. 발굴된...,"대장암,항암제 내성,세포사멸,유전자발굴",0,대장암의 TRAIL 내성 표적 인자 발굴 및 TRAIL 반응 예측 유전자 지도 구축...


In [ ]:
test.head(2)

,과제명,요약문_연구목표,요약문_한글키워드,data
0,R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...,○ 차체 점용접부의 품질 검사를 위한 64채널 무선 기반 C-Scan 탐촉자 개발\...,"마찰교반점용접, 비파괴 검사, 초음파 탐상, 씨 스캔, 용접 품질 평가",R-FSSW 기술 적용 경량 차체 부품 개발 및 품질 평가를 위한 64채널 C-SC...
1,다입자계를 묘사하는 편미분방정식에 대한 연구,자연계에는 입자의 개수가 아주 큰 다양한 다입자계가 존재한다. 이런 다입자계의 효...,"다체계 방정식,동역학의 안정성,양자역학,고전역학,평균장 극한,고전극한,비상대론적 극한",다입자계를 묘사하는 편미분방정식에 대한 연구자연계에는 입자의 개수가 아주 큰 다양한...


# **3. 모델링**

In [ ]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)
BATCH_SIZE = 32
NUM_EPOCHS = 5
VALID_SPLIT = 0.2
MAX_LEN=200

In [ ]:
from transformers import BertTokenizer

In [ ]:
tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased',  cache_dir='bert_ckpt', do_lower_case=False)

def bert_tokenizer(sent, MAX_LEN):
    
    encoded_dict=tokenizer.encode_plus(
    text = sent, 
    add_special_tokens=True, 
    max_length=MAX_LEN, 
    pad_to_max_length=True, 
    return_attention_mask=True,
    truncation = True)
    
    input_id=encoded_dict['input_ids']
    attention_mask=encoded_dict['attention_mask']
    token_type_id = encoded_dict['token_type_ids']
    
    return input_id, attention_mask, token_type_id

input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

for train_sent, train_label in zip(train['data'], train['label']):
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(train_sent), MAX_LEN=MAX_LEN)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
        train_data_labels.append(train_label)
        
    except Exception as e:
        print(e)
        print(train_sent)
        pass

train_input_ids=np.array(input_ids, dtype=int)
train_attention_masks=np.array(attention_masks, dtype=int)
train_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
train_inputs=(train_input_ids, train_attention_masks, train_token_type_ids)
train_labels=np.asarray(train_data_labels, dtype=np.int32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
print(train_input_ids[1])
print(train_attention_masks[1])
print(train_token_type_ids[1])
print(tokenizer.decode(train_input_ids[1]))

[   101   9069  13890 119115  10459   8996  17138   9934  14801   9640
  13764   9323 118654   9316   9321 119187   9576 119281   9625  16617
  13764   9706  12092   8908  70122  10530  42300  91785 119273  22200
  68055  37824   8848  15891  17138   9934  14801   9625  16617  48959
   9323 118654  12453   8996  17138  17730  12965   8932  54918  91785
   9323 118654  13441   9625  16617  48959   9879  11102   8932  30134
   9959 119115  18227  23321  48549  33768   9576 119281   9625  16617
  13764   9706  12092   8908  70122   9995  13764  20626  33077   9095
  29364  31503  86834   9435  55530  31503  86834  10622  25605   9069
  13890 119115  10530   9891  10739  17022   9678  58931  16855  13767
   9428  38631  14801   8996  17138   9625  51431   9625  16617  48959
   9323 118654  12453   9663  50632  17138   8925  70915   8932  54918
   8922  16758   9995  13764  20626  33077   9095  29364  31503  86834
   9435  55530  31503  86834  10622  25605   9069  13890 119115  10530
   989

In [ ]:
class TFBertClassifier(tf.keras.Model):
    def __init__(self, model_name, dir_path, num_class):
        super(TFBertClassifier, self).__init__()

        self.bert = TFBertModel.from_pretrained(model_name, cache_dir=dir_path)
        self.dropout = tf.keras.layers.Dropout(self.bert.config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(num_class, 
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(self.bert.config.initializer_range), 
                                                name="classifier")
        
    def call(self, inputs, attention_mask=None, token_type_ids=None, training=False):
        
        #outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
        outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
        pooled_output = outputs[1] 
        pooled_output = self.dropout(pooled_output, training=training)
        logits = self.classifier(pooled_output)

        return logits

cls_model = TFBertClassifier(model_name='bert-base-multilingual-cased',
                                  dir_path='bert_ckpt',
                                  num_class=46)

# 학습 준비하기
optimizer = tf.keras.optimizers.Adam(3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
cls_model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

model_name = "tf2_bert_classifier"

# overfitting을 막기 위한 ealrystop 추가
earlystop_callback = EarlyStopping(monitor='val_accuracy', min_delta=0.0001,patience=5)
# min_delta: the threshold that triggers the termination (acc should at least improve 0.0001)
# patience: no improvment epochs (patience = 1, 1번 이상 상승이 없으면 종료)\

checkpoint_path = os.path.join(model_name, 'weights.h5')
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create path if exists
if os.path.exists(checkpoint_dir):
    print("{} -- Folder already exists \n".format(checkpoint_dir))
else:
    os.makedirs(checkpoint_dir, exist_ok=True)
    print("{} -- Folder create complete \n".format(checkpoint_dir))
    
cp_callback = ModelCheckpoint(
    checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True)

# 학습과 eval 시작
history = cls_model.fit(train_inputs, train_labels, epochs=2, batch_size=32,
                    validation_split = VALID_SPLIT, callbacks=[earlystop_callback, cp_callback])

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


tf2_bert_classifier -- Folder already exists 

Epoch 1/2
4358/4358 [==============================] - 3565s 814ms/step - loss: 0.6396 - accuracy: 0.8412 - val_loss: 0.4961 - val_accuracy: 0.8602

Epoch 00001: val_accuracy improved from -inf to 0.86024, saving model to tf2_bert_classifier/weights.h5
Epoch 2/2
4358/4358 [==============================] - 3545s 813ms/step - loss: 0.4385 - accuracy: 0.8701 - val_loss: 0.4242 - val_accuracy: 0.8730

Epoch 00002: val_accuracy improved from 0.86024 to 0.87298, saving model to tf2_bert_classifier/weights.h5


In [ ]:
input_ids =[]
attention_masks =[]
token_type_ids =[]
train_data_labels = []

def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

for test_sent in test['data']:
    try:
        input_id, attention_mask, token_type_id = bert_tokenizer(clean_text(test_sent), MAX_LEN=40)
        
        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        #########################################
       
    except Exception as e:
        print(e)
        print(test_sent)
        pass
    
test_input_ids=np.array(input_ids, dtype=int)
test_attention_masks=np.array(attention_masks, dtype=int)
test_token_type_ids=np.array(token_type_ids, dtype=int)
###########################################################
test_inputs=(test_input_ids, test_attention_masks, test_token_type_ids)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2132: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
results = cls_model.predict(test_inputs)
len(results)

43576

In [ ]:
results=tf.argmax(results, axis=1)
len(results)

43576

In [ ]:
sample_submission['label']= results

In [ ]:
sample_submission

,index,label
0,174304,0
1,174305,0
2,174306,0
3,174307,0
4,174308,0
...,...,...
43571,217875,0
43572,217876,0
43573,217877,2
43574,217878,0


In [ ]:
sample_submission.to_csv('bert.csv', index=False)
